Group number: 

Students (name : student number):


1.


2.

3.

4.

5.


## 🚚 BDMM Second Homework Assignment 🚚 

_The Wide World Importers (WWI) is a wholesales novelty goods importer and distributor operating from the San Francisco bay area. In this assignment we will be working with their database._ 
You can get more information and details about the WWI database in the following link: https://docs.microsoft.com/en-us/sql/samples/wide-world-importers-what-is?view=sql-server-ver15

The focus of the second assignment is modelling. We will use the World Wide Importers database and convert it to a document-based database. To that end, we will be leveraging concepts like data denormalization, indices, and mongodb design patterns. 

More information on the extended datamodel to be found here: </br>  
https://docs.microsoft.com/en-us/sql/samples/wide-world-importers-oltp-database-catalog?view=sql-server-ver15

## Problem Description

Your team has just arrived at WWI (a leading company in logistics). Welcome!   <br>
Even though business is thriving, the IT department is going through a bad time.   <br>
Digitalization was never a priority for the company and now the company operational and analytical requirements are starting to grow beyond the capabilities of their existing data architecture.   <br>

WWI data are spread accross different systems, but we've already managed to pull them all into a mongo dump file. This data file is an exact dump of the SQL data so includes all the same structure, the SQL tables become collections and the rows become documents. This means all the original SQL keys are included in the data.<br>
Currently, the costs to develop the necessary queries to collect data to answer questions asked by the different departments are too high. <br>

Management concluded it is the right time to revise and revamp the data architecture, in order to speed up operations. 

In that context, your team was tasked with merging all the company data into a single and coherent Mongo database. <br>
It is expected that, with your solution, WWI will have a better understanding of their business and that the different departments will be able to obtain efficiently the answers they need.

The WWI team shared with you an ERD of their current datamodel:<br>
![datamodel](./WWI.png)

**Note** You can open the file WWI.png that is in the same directory as this notebook to see the above image in more detail and zoom in as you need.

Addtionally, the WWI team asked you the deliver the following outputs in **4 weeks**:
- Understand and model the database in MongoDB.
- Setup the database so that it is performs well for the queries they have provided. You should include reasoning in comments for the decisions you make on modelling the database.
- Answer the questions (queries) on the data provided.  
- Submit the results by following the instructions.  

With these deliveries, you will have created a prototype and allows the management to decide whether MongoDB is a good solution that meets their requirements.

### Design Requirements

Note that WWI has the following query requirements for the database.

1. The web team needs to know:  
    1. Which state province do we have the most suppliers in?  
    2. How many people have three or more `OtherLanguage`? 
    3. Top 10 most common `OtherLanguage` for people records. 
    4. How many customer records are valid after `November 2015`? 
    5. What percentage of people records don't have the UserPreferences field? 

2. The warehouse group needs to know:  
    1. What is the average difference in days between OrderDate and ExpectedDeliveryDate for orders sold by (`SalespersonPersonID`) person with the name `Jack Potter`?
    2. Which items get ordered the most in bulk (largest average quantity ordered)?  
    3. Which two items get ordered together the most?
    4. For each customer category which 3 items have the ordered the most?
    5. What is the current stock of each stockgroup?

Transform the mongo dump file provided with this notebook and model a database following mongodb's best practices. You should adjust the data model to best fit the use cases provided above. Think about collections, embedding, linking, indexing, and the patterns learned in class. Provide justifications for each decision you make. What, if any, are the trade-off's or disadvantages of your approach.

Use MongoDB queries to answer the questions on your transformed database.

### Deliverables

1. Notebook with all DB creation operations and CRUD operations to create the data model. **Important** you should include in comments justification for your decisions on modelling the data.;
2. Second notebook with all required queries and answers for the questions, **Important** please indicate with comments the steps in the data model you took to optimise each query;


# Additional Information

## Groups  

Students should form groups of at least 4 and at most 5. <br>

## Submission  Deadline

The submission includes two notebooks with outputs (cells must be run). 
Please make sure to indicate:
1. group number,
2. group members with student names and numbers,
3. the name of the database that you created. <br>

Upload the notebook on moodle before **23:59 on June 22nd**

## Evaluation   

The second homework assignment counts 40% towards your final mark of the curricular unit. <br>
The assignment will be scored from 0 to 20. <br>

Each group submission will be evaluated on three components:
1. correctness of results;
2. simplicity and performance characteristics of the solution;
3. justification of decisions.

50% -  Database design  
50% -  Query results including performance

Please note that all code delivered in this assignment will go through plagiarism automated checks. <br>
Groups with high similarity levels in their code will undergo investigation.


# Database setup

## Import

In [1]:
#!pip install pymongo

In [3]:
# Python connector
from pymongo import MongoClient
from bson.objectid import ObjectId
from pprint import pprint
import json
from datetime import datetime

user="AzureDiamond"
password="hunter2"
host="localhost"
port="27017"
protocol="mongodb"

client = MongoClient(f"{protocol}://{user}:{password}@{host}:{port}")

In [4]:
db = client.WideWorldImporters
print(f"Database info: {db}\n")

Database info: Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'WideWorldImporters')



In [5]:
client.drop_database("mydatabase")
db_new = client["mydatabase"]

In [6]:
collection_list = db.list_collection_names()
print(f"The database contains {len(collection_list)} collections")

print(f"First 5 collections: {collection_list[0:5]}")

The database contains 25 collections
First 5 collections: ['suppliertransactions', 'stockgroups', 'stockitemstransactions', 'suppliercategories', 'packagetypes']


In [7]:
db.list_collection_names()

['suppliertransactions',
 'stockgroups',
 'stockitemstransactions',
 'suppliercategories',
 'packagetypes',
 'purchaseorderlines',
 'orderlines',
 'orders',
 'stockitemsstockgroups',
 'colors',
 'purchaseorders',
 'deliverymethods',
 'customercategories',
 'customertransactions',
 'invoices',
 'customers',
 'invoicelines',
 'people',
 'cities',
 'stockitems',
 'stateprovinces',
 'countries',
 'paymentmethods',
 'transactiontypes',
 'suppliers']

## Embeddings

In [8]:
db_new.list_collection_names()

[]

In [9]:
# Embedding Customer Categories and Provinces in Customers

query_1 = {
        "$lookup":{
           "from": "customercategories",
           "localField": "CustomerCategoryID",
           "foreignField": "CustomerCategoryID",
           "as": "category"
        }
    }

query_2 = {
        "$lookup":{
           "from": "cities",
           "localField": "PostalCityID",
           "foreignField": "CityID",
           "as": "city"
        }
    }


query_3 = {
        "$lookup":{
           "from": "stateprovinces",
           "localField": "city.StateProvinceID",
           "foreignField": "StateProvinceID",
           "as": "province"
        }
    }


query_4 = {
            "$project": {
                "city": 0,
    }
}

query_5 = { 
        "$merge" : {
            "into" : {"db": "mydatabase", "coll": "customers"},
            "whenMatched": "replace"
        } 
    }

pipeline = [query_1, query_2, query_3, query_4, query_5]

r = db.customers.aggregate(pipeline)

In [10]:
# Embedding StockGroup in StockItems

query_1 = {
        "$lookup":{
           "from": "stockitemsstockgroups",
           "localField": "StockItemID",
           "foreignField": "StockItemID",
           "as": "stockitemgroup"
        }
    }


query_2 = {
        "$lookup":{
           "from": "stockgroups",
           "localField": "stockitemgroup.StockGroupID",
           "foreignField": "StockGroupID",
           "as": "stockgroup"
        }
    }


query_3 = {
            "$project": {
                "stockitemgroup": 0,
    }
}

query_4 = { 
        "$merge" : {
            "into" : {"db": "mydatabase", "coll": "stocks"},
            "whenMatched": "replace"
        } 
    }

pipeline = [query_1, query_2, query_3, query_4]

r = db.stockitems.aggregate(pipeline)

In [11]:
# Embedding Provinces in Suppliers

query_1 = {
        "$lookup":{
           "from": "cities",
           "localField": "PostalCityID",
           "foreignField": "CityID",
           "as": "city"
        }
    }


query_2 = {
        "$lookup":{
           "from": "stateprovinces",
           "localField": "city.StateProvinceID",
           "foreignField": "StateProvinceID",
           "as": "province"
        }
    }


query_3 = {
            "$project": {
                "city": 0,
    }
}

query_4 = { 
        "$merge" : {
            "into" : {"db": "mydatabase", "coll": "suppliers"},
            "whenMatched": "replace"
        } 
    }

pipeline = [query_1, query_2, query_3, query_4]

r = db.suppliers.aggregate(pipeline)

In [12]:
# Embedding Orderlines in Orders
# takes 3 hours

query_1 = {
        "$lookup":{
           "from": "orderlines",
           "localField": "OrderID",
           "foreignField": "OrderID",
           "as": "OrderID"
        }
    }

query_2 = { 
        "$merge" : {
            "into" : {"db": "mydatabase", "coll": "orders"},
            "whenMatched": "replace"
        } 
    }

pipeline = [query_1, query_2]

r = db.orders.aggregate(pipeline)


In [13]:
# Embedding Customers in Orders

query_1 = {
        "$lookup":{
           "from": "customers",
           "localField": "CustomerID",
           "foreignField": "CustomerID",
           "as": "CustomerID"
        }
    }

query_2 = { 
        "$merge" : {
            "into" : {"db": "mydatabase", "coll": "orders"},
            "whenMatched": "replace"
        } 
    }

pipeline = [query_1, query_2]

r = db_new.orders.aggregate(pipeline)

In [14]:
# Embedd people in orders

query_1 = {
        "$lookup":{
           "from": "people",
           "localField": "SalespersonPersonID",
           "foreignField": "PersonID",
           "as": "Salesperson"
        }
    }

query_2 = { 
        "$merge" : {
            "into" : {"db": "mydatabase", "coll": "orders"},
            "whenMatched": "replace"
        } 
    }

pipeline = [query_1, query_2]

r = db_new.orders.aggregate(pipeline)

In [15]:
# Embedded Invoicelines in Invoice

query_1 = {
        "$lookup":{
           "from": "invoicelines",
           "localField": "InvoiceID",
           "foreignField": "InvoiceID",
           "as": "InvoiceID"
        }
    }

query_2 = { 
        "$merge" : {
            "into" : {"db": "mydatabase", "coll": "invoices"},
            "whenMatched": "replace"
        } 
    }

pipeline = [query_1, query_2]

r = db.invoices.aggregate(pipeline)

AutoReconnect: localhost:27017: connection closed

In [17]:
# Take over the peoples collection in the new database

query_1 = { 
        "$merge" : {
            "into" : {"db": "mydatabase", "coll": "people"},
            "whenMatched": "replace"
        } 
    }

r = db.people.aggregate([query_1])

# Change type of "OtherLanguages" to JSON Object

# for person in db_new.people.find():
#     lang = person["OtherLanguages"]
#     if lang == None:
#         continue
#     lang = json.loads(lang)
#     filter = {"PersonID" : person["PersonID"]}
#     newvalues = { "$set": { 'OtherLanguages': lang } }
#     db_new.people.update_one(filter, newvalues)

In [18]:
# Take over the stockitemstransactions collection in the new database and embedding stockitems in it

query_1 = { 
        "$merge" : {
            "into" : {"db": "mydatabase", "coll": "stockitemstransactions"},
            "whenMatched": "replace"
        } 
    }

r = db.stockitemstransactions.aggregate([query_1])

query_1 = {
        "$lookup":{
           "from": "stocks",
           "localField": "StockItemID",
           "foreignField": "StockItemID",
           "as": "Item"
        }
    }

query_2 = { 
        "$merge" : {
            "into" : {"db": "mydatabase", "coll": "stockitemstransactions"},
            "whenMatched": "replace"
        } 
    }

pipeline = [query_1, query_2]

r = db_new.stockitemstransactions.aggregate(pipeline)

## References

# Queries

## The web team needs to know:

In [19]:
# Which state province do we have the most suppliers in?

result = list(db_new.suppliers.aggregate([ 

    {'$group':{'_id':{'Province':'$province.StateProvinceName'}, 'Count':{'$sum':1}}}, 
    
    {'$sort':{'Count' : -1}},
    
    {'$limit' : 1}
]))

result

[{'_id': {'Province': ['California']}, 'Count': 3}]

In [20]:
# How many people have three or more OtherLanguage?

result = list(db_new.people.aggregate([

    {'$match': {
        "OtherLanguages" : {"$not" : {"$type" : 10}}
    }},

    { "$unwind": {
        "path": "$OtherLanguages",
        "preserveNullAndEmptyArrays": False
    } },
{
    "$group": {
        "_id": "$PersonID",
        "Count": {
            "$sum": 1
        }
    }
},
{
    "$match": {
        "Count" :  {"$gte":3}
    }
},
]))

pprint(result)

[{'Count': 3, '_id': 2},
 {'Count': 3, '_id': 7},
 {'Count': 3, '_id': 10},
 {'Count': 3, '_id': 8}]


In [21]:
# Top 10 most common OtherLanguage for people records.

result = list(db_new.people.aggregate([

    {'$match': {
        "OtherLanguages" : {"$not" : {"$type" : 10}}
    }},

    { "$unwind": {
        "path": "$OtherLanguages",
        "preserveNullAndEmptyArrays": False
    } },
{
    "$group": {
        "_id": "$OtherLanguages",
        "Count": {
            "$sum": 1
        }
    }
},
{
    "$sort": {
        "Count" : -1
    }
},
    
{   "$limit":10
    }
    
]))

pprint(result)

[{'Count': 3, '_id': 'Greek'},
 {'Count': 3, '_id': 'Dutch'},
 {'Count': 3, '_id': 'Finnish'},
 {'Count': 2, '_id': 'Slovak'},
 {'Count': 2, '_id': 'Arabic'},
 {'Count': 2, '_id': 'Croatian'},
 {'Count': 2, '_id': 'Lithuanian'},
 {'Count': 2, '_id': 'Polish'},
 {'Count': 2, '_id': 'Romanian'},
 {'Count': 1, '_id': 'Estonian'}]


In [22]:
# How many customer records are valid after November 2015?

dt = datetime(2015, 11, 1)

result = list(db_new.customers.aggregate([

    {"$match" : {"ValidTo" : {
        "$gt" : dt}}},
    {
      "$count": "CustomerID"
    }
]))

pprint(result)

[{'CustomerID': 663}]


In [23]:
# What percentage of people records don't have the UserPreferences field?

result_all = list(db_new.people.aggregate([
     {
        "$count": "PersonID"
     }
    ]))

result_all = result_all[0]["PersonID"]


result = list(db_new.people.aggregate([
   
    {'$match': {
        "UserPreferences" : {"$type" : 10}}},
    {
        "$group": {
                "_id": "$Null",
                "count": {
                    "$sum": 1
            }}}
    ]))
    
result = result[0]["count"]

print(result/result_all*100)

83.61836183618362


## The warehouse group needs to know:

In [33]:
# What is the average difference in days between OrderDate and ExpectedDeliveryDate for orders sold by (SalespersonPersonID) person with the name Jack Potter?

result = list(db_new.orders.aggregate([

    {"$match" : {"Salesperson.FullName" : "Jack Potter"}},
    
.
 ]))

pprint(result)

[]


In [31]:
db_new.orders.find_one()

{'_id': ObjectId('6287c5ec636e5a126944ceae'),
 'OrderID': [{'_id': ObjectId('6287c5d0636e5a126941469a'),
   'OrderLineID': 2,
   'OrderID': 1,
   'StockItemID': 67,
   'Description': 'Ride on toy sedan car (Black) 1/12 scale',
   'PackageTypeID': 7,
   'Quantity': 10,
   'UnitPrice': Decimal128('230.00'),
   'TaxRate': Decimal128('15.000'),
   'PickedQuantity': 10,
   'PickingCompletedWhen': datetime.datetime(2013, 1, 1, 11, 0),
   'LastEditedBy': 3,
   'LastEditedWhen': datetime.datetime(2013, 1, 1, 11, 0)}],
 'CustomerID': [{'_id': ObjectId('6287c598636e5a12693dc9c8'),
   'CustomerID': 832,
   'CustomerName': 'Aakriti Byrraju',
   'BillToCustomerID': 832,
   'CustomerCategoryID': 4,
   'BuyingGroupID': None,
   'PrimaryContactPersonID': 3032,
   'AlternateContactPersonID': None,
   'DeliveryMethodID': 3,
   'DeliveryCityID': 13242,
   'PostalCityID': 13242,
   'CreditLimit': Decimal128('3500.00'),
   'AccountOpenedDate': datetime.datetime(2013, 1, 1, 0, 0),
   'StandardDiscountPercen

In [164]:
# Which items get ordered the most in bulk (largest average quantity ordered)?

# unwind orderlines, group by product, average quantity

result = list(db_new.orders.aggregate([
    
    {"$unwind" : "$OrderID"},
    
    {'$group':{'_id':{'ProductDescription': '$OrderID.Description', "ProductID":"$OrderID.StockItemID"}, 'AvgQuantity':{'$avg': "$OrderID.Quantity"}}},
    
    {"$sort" : {"AvgQuantity" : -1}},
    
    {"$limit" : 5}

]))

pprint(result)

[{'AvgQuantity': 199.35,
  '_id': {'ProductDescription': 'Black and orange fragile despatch tape '
                                '48mmx75m',
          'ProductID': 191}},
 {'AvgQuantity': 198.23950870010236,
  '_id': {'ProductDescription': 'Black and orange fragile despatch tape '
                                '48mmx100m',
          'ProductID': 192}},
 {'AvgQuantity': 145.26190476190476,
  '_id': {'ProductDescription': 'Clear packaging tape 48mmx75m',
          'ProductID': 189}},
 {'AvgQuantity': 141.64338919925513,
  '_id': {'ProductDescription': 'Shipping carton (Brown) 356x356x279mm',
          'ProductID': 185}},
 {'AvgQuantity': 141.48096564531104,
  '_id': {'ProductDescription': '3 kg Courier post bag (White) 300x190x95mm',
          'ProductID': 188}}]


In [22]:
# Which two items get ordered together the most?

result = list(db_new.orders.aggregate([
    
    {"$unwind" : "$OrderID"},
    
    

]))

pprint(result)

In [1]:
db_new.orders.find_one()

NameError: name 'db_new' is not defined

In [180]:
# For each customer category which 3 items have the ordered the most?
# To answer the question we assumed the number of orders per product. Alternatively the Quantity could have been used
# by changing the "$sum" argument from 1 to "$OrderID.Quantity"

result = list(db_new.orders.aggregate([
    
    {"$unwind" : "$OrderID"},
    
    {'$group':{'_id':{'CustomerCategory':'$CustomerID.category.CustomerCategoryID', "StockItemID" : "$OrderID.Description"}, 'Count':{'$sum':1}}},
    
    # Due to Mongo DB Version 5.0 (and not 5.2) we were not able to use the $topN below, which would have been the cleaner solution.
    
    #{'$group':{'_id': '$CustomerCategory', "StockItems" : {"$topN" : {"n" : 3, "sortBy" : {"Count":-1}, "output" : "$$ROOT"}}}},
    
    {"$sort" : {"Count" : -1}},
    
    {"$group": {"_id" : {"Category" : "$_id.CustomerCategory"}, "StockItems" : {"$push" : "$$ROOT"}}}, 
    
    {"$project" : {"_id":1, "StockItems" : {"$slice" : ["$StockItems", 3]}}}

]))

pprint(result)

[{'StockItems': [{'Count': 110,
                  '_id': {'CustomerCategory': [[4]],
                          'StockItemID': '"The Gu" red shirt XML tag t-shirt '
                                         '(Black) 6XL'}},
                 {'Count': 106,
                  '_id': {'CustomerCategory': [[4]],
                          'StockItemID': '"The Gu" red shirt XML tag t-shirt '
                                         '(White) 7XL'}},
                 {'Count': 104,
                  '_id': {'CustomerCategory': [[4]],
                          'StockItemID': 'Superhero action jacket (Blue) '
                                         'XXS'}}],
  '_id': {'Category': [[4]]}},
 {'StockItems': [{'Count': 95,
                  '_id': {'CustomerCategory': [[6]],
                          'StockItemID': '"The Gu" red shirt XML tag t-shirt '
                                         '(White) 3XS'}},
                 {'Count': 91,
                  '_id': {'CustomerCategory': [[6]],
         

In [ ]:
# What is the current stock of each stockgroup?

result = list(db_new.stockitemstransactions.aggregate([
    
    {"$unwind" : "$Item.stockgroup"},
    
    {"$group" : {"StockGroup" : "$Item.stockgroup.StockGroupID"}, "Current Stock" : {"$sum" : "$Quantity"}}

]))

pprint(result)

In [175]:
db_new.stockitemstransactions.find_one()

{'_id': ObjectId('6287c64d636e5a126945f099'),
 'StockItemTransactionID': 6293,
 'StockItemID': 213,
 'TransactionTypeID': 10,
 'CustomerID': 28,
 'InvoiceID': 1325,
 'SupplierID': None,
 'PurchaseOrderID': None,
 'TransactionOccurredWhen': datetime.datetime(2013, 1, 26, 12, 0),
 'Quantity': Decimal128('-10.000'),
 'LastEditedBy': 4,
 'LastEditedWhen': datetime.datetime(2013, 1, 26, 12, 0),
 'Item': [{'_id': ObjectId('6287c78f636e5a12694aa9fb'),
   'StockItemID': 213,
   'StockItemName': 'Air cushion film 200mmx100mm 325m',
   'SupplierID': 7,
   'ColorID': None,
   'UnitPackageID': 7,
   'OuterPackageID': 7,
   'Brand': None,
   'Size': '325m',
   'LeadTimeDays': 14,
   'QuantityPerOuter': 1,
   'IsChillerStock': False,
   'Barcode': None,
   'TaxRate': Decimal128('15.000'),
   'UnitPrice': Decimal128('87.00'),
   'RecommendedRetailPrice': Decimal128('130.07'),
   'TypicalWeightPerUnit': Decimal128('5.000'),
   'MarketingComments': None,
   'InternalComments': None,
   'Photo': None,
 